In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import joblib
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [41]:
df = pd.read_csv('depression.csv')
df.head()

,Q3,Q5,Q10,Q13,Q16,Q17,Q21,Q24,Q26,Q31,Q34,Q37,Q38,Q42,Classes
0,1,3,0,3,3,2,0,3,3,3,2,0,1,3,3
1,1,3,1,3,2,3,1,1,2,1,1,3,1,1,3
2,3,3,3,3,3,3,3,3,0,3,3,3,3,3,4
3,1,2,2,0,1,2,0,0,1,2,3,1,0,1,2
4,2,3,2,3,2,3,2,1,3,2,3,2,2,2,4


In [42]:
X = df.drop('Classes', axis=1)
y = df.iloc[:, -1:]
print("X shape->",X.shape, "y shape->", y.shape)
y.head()

X shape-> (39775, 14) y shape-> (39775, 1)


,Classes
0,3
1,3
2,4
3,2
4,4


In [18]:
# class_mapping = {0: 'N', 1: 'M', 2: 'MD', 3: 'S', 4: 'ES'}
# y= y.replace(class_mapping)
# y.head()

In [19]:
# info_gain = mutual_info_classif(X, y)

# # Create a DataFrame to store feature names and their information gain scores
# feature_info_gain = pd.DataFrame({'Feature': X.columns, 'Information Gain': info_gain})

# # Sort features based on information gain
# feature_info_gain = feature_info_gain.sort_values(by='Information Gain', ascending=False)

# # Print information gain
# print(feature_info_gain)

# # Plot the information gain scores
# plt.figure(figsize=(10, 6))
# plt.barh(feature_info_gain['Feature'], feature_info_gain['Information Gain'], color='skyblue')
# plt.xlabel('Information Gain')
# plt.title('Feature Importance (Information Gain)')
# plt.show()

In [20]:
# X = X.drop(labels=['Q19', 'Q23', 'Q2'], axis=1)
# X = X.drop(labels=['Q2'], axis=1)
# X.head()

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [22]:
print("Before Over Sampling\n", y_train.value_counts())
sampler = RandomOverSampler()
X_train_oversampled, y_train_oversampled = sampler.fit_resample(X_train, y_train)
print("\nAfter Over Sampling\n", y_train_oversampled.value_counts())

Before Over Sampling
 Classes
4          10830
0           7076
2           5651
3           5202
1           3061
Name: count, dtype: int64

After Over Sampling
 Classes
0          10830
1          10830
2          10830
3          10830
4          10830
Name: count, dtype: int64


**Decision Tree**

In [23]:
params_dt = {'criterion': ['gini', 'entropy'],
          'max_depth': [None] + [i for i in range(1, 11)]}

In [24]:
model_dt = DecisionTreeClassifier()
grid_search_dt = GridSearchCV(estimator=model_dt, param_grid=params_dt, cv=10, scoring='accuracy')

In [25]:
grid_search_dt.fit(X_train.values, y_train.values)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='accuracy')

In [26]:
grid_search_dt.best_params_

{'criterion': 'entropy', 'max_depth': None}

In [27]:
grid_search_dt.best_score_

0.8370521684475174

In [28]:
best_model_dt = grid_search_dt.best_estimator_
y_pred = best_model_dt.predict(X_test.values)
clf_report = classification_report(y_test, y_pred)
print(clf_report)
joblib.dump(value=best_model_dt, filename='Decision_Tree.joblib')

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1780
           1       0.61      0.63      0.62       725
           2       0.75      0.74      0.75      1428
           3       0.72      0.70      0.71      1275
           4       0.93      0.94      0.94      2747

    accuracy                           0.84      7955
   macro avg       0.79      0.79      0.79      7955
weighted avg       0.84      0.84      0.84      7955



['Decision_Tree.joblib']

**Random Forest**

In [29]:
best_model_rf = RandomForestClassifier(criterion='gini', n_estimators=1000)
best_model_rf.fit(X_train.values, y_train.values)
y_pred_rf = best_model_rf.predict(X_test.values)
print(classification_report(y_test, y_pred_rf))
joblib.dump(value=best_model_rf, filename='Random_Forest.joblib')

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1780
           1       0.84      0.80      0.82       725
           2       0.91      0.90      0.91      1428
           3       0.90      0.89      0.90      1275
           4       0.97      0.98      0.97      2747

    accuracy                           0.93      7955
   macro avg       0.91      0.91      0.91      7955
weighted avg       0.93      0.93      0.93      7955



['Random_Forest.joblib']

**SVM**

In [30]:
svc = SVC(probability=True)
svc.fit(X_train.values, y_train.values)
y_pred_svm = svc.predict(X_test.values)
print(classification_report(y_test, y_pred_svm))
joblib.dump(value=svc, filename='SVM.joblib')

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1780
           1       1.00      1.00      1.00       725
           2       1.00      1.00      1.00      1428
           3       1.00      1.00      1.00      1275
           4       1.00      1.00      1.00      2747

    accuracy                           1.00      7955
   macro avg       1.00      1.00      1.00      7955
weighted avg       1.00      1.00      1.00      7955



['SVM.joblib']

**MLP Classifier**

In [31]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
print(classification_report(y_test, y_pred_mlp))
joblib.dump(value=mlp, filename='MLP.joblib')

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1780
           1       1.00      1.00      1.00       725
           2       1.00      1.00      1.00      1428
           3       1.00      1.00      1.00      1275
           4       1.00      1.00      1.00      2747

    accuracy                           1.00      7955
   macro avg       1.00      1.00      1.00      7955
weighted avg       1.00      1.00      1.00      7955



['MLP.joblib']

**KNN**

In [32]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print(classification_report(y_test, y_pred_knn))

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1780
           1       0.82      0.73      0.77       725
           2       0.89      0.88      0.89      1428
           3       0.89      0.86      0.87      1275
           4       0.97      0.97      0.97      2747

    accuracy                           0.92      7955
   macro avg       0.90      0.89      0.89      7955
weighted avg       0.92      0.92      0.92      7955



**Naive Bayes**

In [33]:
gNB = GaussianNB()
gNB.fit(X_train, y_train)
y_pred_gNB = gNB.predict(X_test)
print(classification_report(y_test, y_pred_gNB))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93      1780
           1       0.66      0.87      0.75       725
           2       0.89      0.88      0.89      1428
           3       0.72      0.96      0.82      1275
           4       1.00      0.86      0.92      2747

    accuracy                           0.88      7955
   macro avg       0.86      0.89      0.86      7955
weighted avg       0.91      0.88      0.89      7955



C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
